In [6]:
import osmnx as ox
import folium
from shapely.geometry import shape
import sys

sys.path.append("../src/")
from utils import (
    get_nearest_station,
    get_valencian_open_data,
    get_gdf,
)

from routes import get_route

cycling_graph = ox.load_graphml("../data/valencia_cycling_network.graphml")
walking_graph = ox.load_graphml("../data/valencia_walking_network.graphml")

In [7]:
def get_valenbisi_route(start, end, cycling_graph, walking_graph, valenbisi_stations):
    """
    Get the Valenbisi route from start to end using the cycling network graph.

    Parameters:
        start(tuple): Tuple of coordinates for the start point.
        end(tuple): Tuple of coordinates for the end point.
        cycling_graph: The cycling network graph.
        walking_graph: The walking network graph.
        valenbisi_stations(geoDataFrame): A GeoDataFrame containing Valenbisi stations.

    Returns:
        tuple: A tuple containing three lists:
            - The walking route from the start to the nearest Valenbisi station.
            - The cycling route between the two Valenbisi stations.
            - The walking route from the nearest Valenbisi station to the end point.
            - The nearest Valenbisi station to the start point.
            - The nearest Valenbisi station to the end point.
    """

    threshold = 0.0001

    cycling_ini_node = ox.distance.nearest_nodes(cycling_graph, X=start[1], Y=start[0])

    cycling_end_node = ox.distance.nearest_nodes(cycling_graph, X=end[1], Y=end[0])

    ini_valenbisi_station = valenbisi_stations[
        valenbisi_stations["available"] > 0
    ].copy()
    end_valenbisi_station = valenbisi_stations[valenbisi_stations["free"] > 0].copy()

    ini_station = get_nearest_station(
        (
            cycling_graph.nodes[cycling_ini_node]["y"],
            cycling_graph.nodes[cycling_ini_node]["x"],
        ),
        ini_valenbisi_station,
    )
    end_station = get_nearest_station(
        (
            cycling_graph.nodes[cycling_end_node]["y"],
            cycling_graph.nodes[cycling_end_node]["x"],
        ),
        end_valenbisi_station,
    )

    ini_station_loc = ini_station["geo_point_2d"]
    end_station_loc = end_station["geo_point_2d"]

    ini_walking_route, dist1 = get_route(
        start, (ini_station_loc["lat"], ini_station_loc["lon"]), walking_graph
    )
    end_walking_route, dist2 = get_route(
        (end_station_loc["lat"], end_station_loc["lon"]), end, walking_graph
    )
    cycling_route, dist3 = get_route(
        (ini_station_loc["lat"], ini_station_loc["lon"]),
        (end_station_loc["lat"], end_station_loc["lon"]),
        cycling_graph,
    )

    dist_ini_station = ox.distance.euclidean(
        ini_station_loc["lat"],
        ini_station_loc["lon"],
        cycling_graph.nodes[cycling_route[0]]["y"],
        cycling_graph.nodes[cycling_route[0]]["x"],
    )

    dist_end_station = ox.distance.euclidean(
        end_station_loc["lat"],
        end_station_loc["lon"],
        cycling_graph.nodes[cycling_route[-1]]["y"],
        cycling_graph.nodes[cycling_route[-1]]["x"],
    )

    if dist_ini_station > threshold:
        inter_ini, d_aux = get_route(
            (ini_station_loc["lat"], ini_station_loc["lon"]),
            (
                cycling_graph.nodes[cycling_route[0]]["y"],
                cycling_graph.nodes[cycling_route[0]]["x"],
            ),
            walking_graph,
        )
        ini_walking_route.extend(inter_ini)
        dist1 += d_aux
    if dist_end_station > threshold:
        inter_end, d_aux = get_route(
            (
                cycling_graph.nodes[cycling_route[-1]]["y"],
                cycling_graph.nodes[cycling_route[-1]]["x"],
            ),
            (end_station_loc["lat"], end_station_loc["lon"]),
            walking_graph,
        )
        end_walking_route = inter_end + end_walking_route
        dist2 += d_aux
    return (
        ini_walking_route,
        dist1,
        cycling_route,
        dist2,
        end_walking_route,
        dist3,
        ini_station,
        end_station,
    )


In [8]:
def print_route(route, graph, map, color="blue"):
    """
    Print the route on the map.

    Parameters:
        route (list): List of nodes in the route.
        graph: The graph containing the nodes.
        map: The folium map to draw the route on.
        color (str): Color of the route line on the map.
    """
    if not route:
        print("No route found.")
        return

    for i in range(len(route) - 1):
        edge_data = graph.get_edge_data(route[i], route[i + 1])
        if edge_data and "geometry" in edge_data[0]:
            geom = edge_data[0]["geometry"]
            folium.PolyLine(
                locations=[(point[1], point[0]) for point in geom.coords],
                color=color,
                weight=5,
            ).add_to(map)
        else:
            folium.PolyLine(
                locations=[
                    (
                        graph.nodes[route[i]]["y"],
                        graph.nodes[route[i]]["x"],
                    ),
                    (
                        graph.nodes[route[i + 1]]["y"],
                        graph.nodes[route[i + 1]]["x"],
                    ),
                ],
                color=color,
                weight=5,
            ).add_to(map)

In [9]:
def print_stations(ini, end, map):
    ini_station_loc = ini["geo_point_2d"]
    end_station_loc = end["geo_point_2d"]
    folium.Marker(
        location=(ini_station_loc["lat"], ini_station_loc["lon"]),
        popup=f"Start Station.<br>Available Bikes: {ini['available']}",
        icon=folium.Icon(color="green", icon="bicycle", prefix="fa"),
    ).add_to(map)

    folium.Marker(
        location=(end_station_loc["lat"], end_station_loc["lon"]),
        popup=f"End Station: {end['address']}<br>Available Places: {end['free']}",
        icon=folium.Icon(color="red", icon="home"),
    ).add_to(map)

In [10]:
# from routes import get_valenbisi_route, print_route, print_stations

params = {
    "rows": 100,
}
url = "https://valencia.opendatasoft.com//api/explore/v2.1/catalog/datasets/valenbisi-disponibilitat-valenbisi-dsiponibilidad/records"
valenbisi_stations = get_valencian_open_data(url, params)

valenbisi_stations = valenbisi_stations[valenbisi_stations["open"] == "T"]
valenbisi_stations["geometry"] = valenbisi_stations["geo_shape"].apply(shape)
valenbisi_stations = get_gdf(valenbisi_stations)

start = (39.470500, -0.385655)  # Example start coordinates (Valencia city center)
end = (39.451831, -0.364881)  # Example end coordinates (nearby location)


(
    ini_walking_route,
    dist_ini,
    cycling_route,
    dist_cycling,
    end_walking_route,
    dist_end,
    ini_station,
    end_station,
) = get_valenbisi_route(start, end, cycling_graph, walking_graph, valenbisi_stations)

m = folium.Map(location=start, zoom_start=14)


# Dibujar las rutas en el mapa
print_route(ini_walking_route, walking_graph, m, color="green")
print_route(cycling_route, cycling_graph, m, color="blue")
print_route(end_walking_route, walking_graph, m, color="green")


# Salida y llegada
folium.Marker(
    location=(start[0], start[1]),
    popup="Start",
    icon=folium.Icon(color="blue"),
).add_to(m)

folium.Marker(
    location=(end[0], end[1]),
    popup="End",
    icon=folium.Icon(color="blue"),
).add_to(m)

# Cambiar el marquer de forma que muestre una bici y un número de bicis en el propio marcador

print_stations(ini_station, end_station, m)

m

In [11]:
cycling_nearest_node

NameError: name 'cycling_nearest_node' is not defined

In [ ]:
cycling_graph[cycling_nearest_node]

AdjacencyView({4579851916: {0: {'osmid': [1314717832, 1314717833, 1314717834, 1314717835, 1314717836, 1314717837, 1314717838, 1314717839, 1314717840, 1314717841, 1314717842, 1314717843, 1314717844, 1314717845, 1314717846, 1314717847, 1314717848, 1314717849, 1314717850, 1314717851, 462608048], 'highway': 'cycleway', 'lanes': '2', 'oneway': False, 'width': '2', 'reversed': False, 'length': 1097.250204775664, 'geometry': <LINESTRING (-0.384 39.471, -0.384 39.471, -0.384 39.472, -0.384 39.472, -0....>}}, 229330299: {0: {'osmid': [1314704672, 1314704673, 1314704674, 462608048, 1314704671], 'highway': 'cycleway', 'lanes': '2', 'oneway': False, 'width': '2', 'reversed': True, 'length': 47.95419387586232, 'geometry': <LINESTRING (-0.384 39.471, -0.384 39.471, -0.384 39.471, -0.384 39.47, -0.3...>}}, 4579851920: {0: {'osmid': 462608056, 'highway': 'cycleway', 'oneway': False, 'reversed': True, 'length': 14.291166783703307, 'geometry': <LINESTRING (-0.384 39.471, -0.384 39.471, -0.383 39.471)>}}

In [ ]:
# buscar las aristas que tengan de fin el nodo cycling_nearest_node
edges = cycling_graph.edges([cycling_nearest_node], data=True)
edges = [(u, v, data) for u, v, data in edges if v == cycling_nearest_node]
# Mostrar las aristas encontradas
for u, v, data in edges:
    print(f"Edge from {u} to {v} with data: {data}")

In [ ]:
# eliminar el nodo del grafo
cycling_graph.remove_node(cycling_nearest_node)

# ox.save_graphml(cycling_graph, "../data/valencia_cycling_sombra.graphml")

In [ ]:
# edge_data = cycling_graph.get_edge_data(cycling_route[0], cycling_route[1])

# if edge_data and "geometry" in edge_data[0]:
#     geom = edge_data[0]["geometry"]
#     for point in geom.coords:
#         print(point)

